In [1]:
from typing import List, Dict
import pydantic, dotenv, os, re

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

LLM = ChatOpenAI(
    model="o4-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

## 1 - Chargement des Documents

In [2]:
all_text = ""

for doc in os.listdir("data/docs-md"):
    if doc.endswith(".md"):
        with open(os.path.join("data/docs-md", doc), "r") as file:
            all_text += file.read()
            
# remove all that starts with "![image](" and ends with ")"
all_text = re.sub(r"!\[image\]\(.*?\)", "", all_text)
            
len(all_text)

43494

In [3]:
MOTS_CLES: Dict[str, str] = {}
DOCS: List[str] = []
KWMAPPING: Dict[str, List[str]] = {}

## 0 - Génération des Mots clés
à partir des documents disponibles

In [4]:
class Explanation(pydantic.BaseModel):
    topic: str = pydantic.Field(description="User assistance topic in the corpus. e. g. 'network' for network issues, 'hardware' for hardware issues, etc.")
    explanation: str = pydantic.Field(description="explanation of the topic, notes on how it might be useful knowledge to solve typical user problems.")

class Analyze(pydantic.BaseModel):
    explanations: List[Explanation]
    
# keywords: Analyze = LLM.with_structured_output(Analyze).invoke(all_text)
# keywords

## 2 - Compression des Documents

In [5]:
class Document(pydantic.BaseModel):
    thematique: str = pydantic.Field(description="Thématique du document synthétique, e. g. 'mots de passe', 'reseaux', etc.")
    contenu: str = pydantic.Field(description="Contenu du document synthétique")

class Documents(pydantic.BaseModel):
    documents: List[Document] = pydantic.Field(description="Nouveaux documents synthétiques qui contiennent toute l'information")
    
messages = [
    SystemMessage(content="""
        Rédige de nouveaux documents (chaque string de la liste est un document) 
        en utilisant toutes les informations des documents que l'utilisateur fournira.
        Regroupez les informations logiquement dans les nouveaux documents.
        Toute information contenue dans le texte de l'utilisateur doit être présent dans au moins un des nouveaux documents.
        Tous les liens doivent être présents avec explication de leur utilité en particulier pour porter assistance à un utilisateur.
        De même, toutes les informations énumérées, les listes, etc doivent être présentes dans les nouveaux documents.
        Les nouveaux documents doivent adopter un format aussi synthétique que possible (par exemple: "politique mots de passe caractères autorisés a b c . / * mais pas ! ? ;").
        Essaie de répartir équitablement l'information en documents qui font au maximum deux paragraphes chacun, moins si possible.
    """),
    HumanMessage(content=str(all_text)),
]

result: Documents = LLM.with_structured_output(Documents).invoke(messages)
DOCS = [d.contenu for d in result.documents]
DOCS

["Changer le mot de passe UTC : https://comptes.utc.fr/accounts-web/tools/changePassword.xhtml (outil officiel). Recommandations ANSSI (http://www.ssi.gouv.fr/IMG/pdf/NP_MDP_NoteTech.pdf) :\n* 8–16 caractères : majuscules, minuscules, chiffres et symboles autorisés (. / * etc.), mais interdits < > ; ` : & % * ? / \\ { } |.\n* Ni mot du dictionnaire, ni nom propre ou marque, ni élément d'identité (nom, date), ni exemple internet.\n* Ne jamais écrire, stocker ni envoyer par mail.\n\nConseils : choisir de préférence 8 caractères, aléatoires ou mnémoniques (phrase personnelle, premières lettres). Garder le secret.",
 'Changer le mot de passe de session Windows : appuyer simultanément Ctrl+Alt+Suppr (voir docs 845/846/847). Saisir l’ancien et le nouveau mot de passe, valider.\n\nMettre à jour le mot de passe aux imprimantes et partages réseau (stocké localement dans le Gestionnaire d’identification) :\n1. Panneau de configuration > Gestionnaire d’identification (doc 848/849).\n2. Sélectionn

## 3 Création d'une prompt de mots clés

In [6]:
keyword_prompt = LLM.invoke([
    SystemMessage(
        content="D'après les informations qui leur sont disponibles pour traiter des requêtes utilisateur, "
                + "propose un grand nombre de mots clés au format lower+dash (= Login refusé => login-refuse) "
                + "qui correspondent à des \"plaintes\" "
                + "que les utilisagers sont susceptibles de porter au service informatique de l'UTC "
                + "(catégories de plaintes, par leur forme, par exemple \"pas-d-internet\" ou "
                + "\"comment-se-connecter\"). Explique les thématiques qu'un système RAG pourrait renvoyer "
                + "suite à l'input de chaque mot clé ou groupe de mots clés."
                + f"\n\nVoici la liste des informations disponibles: {str(DOCS)}"
    )
]).content
keyword_prompt

'Voici une proposition d’une quarantaine de mots-clés (format lower+dash) couvrant les plaintes/types de demandes que les usagers peuvent adresser au service informatique de l’UTC, classés en grandes thématiques, et pour chacune la nature des réponses qu’un système RAG pourrait fournir.\n\n1. Mot-clés « authentification/password »  \n   - changer-mdp-utc  \n   - mdp-oublié-utc  \n   - mdp-non-accepté  \n   - sécurité-mot-de-passe  \n   - mdp-windows  \n   - accès-session-windows  \n\n   RAG pourrait renvoyer :  \n   • un lien et guide pas-à-pas vers l’outil officiel https://comptes.utc.fr/.../changePassword.xhtml  \n   • les recommandations ANSSI (longueur, complexité, caractères interdits)  \n   • procédure Ctrl+Alt+Suppr et références docs 845/846/847  \n   • conseils mnémoniques et bonnes pratiques de stockage.\n\n2. Mot-clés « mise à jour des credentials sur postes et imprimantes »  \n   - maj-mdp-imprimantes  \n   - maj-mdp-partages  \n\n   RAG pourrait renvoyer :  \n   • pas-à-pa

## 4 - Attribution des Mots clés

In [7]:
class KeywordMapping(pydantic.BaseModel):
    document_id: int
    resume: str = pydantic.Field(description="Résumé du document et pourquoi tu as choisis ces mots clés.")
    mots_cles: List[str] = pydantic.Field(description="Liste des mots clés associés aux problèmes que peuvent résoudre ce document.")

class Keywords(pydantic.BaseModel):
    mapping: List[KeywordMapping]
    
messages = [
    SystemMessage(content="""
        Associe des mots clés disponibles aux documents.
        Il faut que lorsqu'un utilisateur tape un mot clé particulier, il trouve tous les documents qui pourraient lui être utiles.
        N'hésite pas à associer plusieurs mots clés à un même document dès qu'il contient la moindre information qui pourraient être partinentes.
        
        Mots clés disponibles (reformule ces mots clés au format lower+dash = Login refusé => login-refuse) :
        'Voici une proposition de catégorisation de plaintes (mots-clés/synonymes) que les utilisateurs peuvent remonter au service IT, ainsi que les thématiques/documents qu’un système RAG pourrait suggérer pour chaque cas.\n\n1. Authentification et accès  \n   Mots-clés/phrases :  \n     • “mot de passe oublié” / “login refusé” / “identifiants invalides”  \n     • “accès refusé” / “permission denied”  \n     • “blocage compte” / “verrouillage session”  \n   Thématiques RAG suggérées :  \n     – Gestion des mots de passe (changement, recommandations ANSSI)  \n     – Déclaration et gestion des comptes UTC  \n     – Récupération et réinitialisation d’identifiants  \n\n2. Réseau et connectivité  \n   Mots-clés/phrases :  \n     • “pas d’internet” / “aucun réseau” / “déconnecté”  \n     • “Wi-Fi ne s’affiche pas” / “connexion eduroam”  \n     • “VPN ne démarre pas” / “erreur OpenVPN / GlobalProtect”  \n   Thématiques RAG suggérées :  \n     – VPN, Wi-Fi et filaire (profils, ports, SSID)  \n     – Dépannage réseau de base (DNS, MAC, DHCP)  \n     – Configuration manuelle de DNS  \n\n3. Partages et stockage  \n   Mots-clés/phrases :  \n     • “lecteur réseau inaccessible” / “montage SMB échoue”  \n     • “SFTP / FileZilla” / “téléversement impossible”  \n     • “droits écriture/lecture”  \n   Thématiques RAG suggérées :  \n     – Accès aux fichiers et lecteurs réseau (SMB, SFTP)  \n     – Activation du service SSH/SFTP  \n     – Gestionnaire d’identification Windows  \n\n4. Messagerie  \n   Mots-clés/phrases :  \n     • “envoi mail échoue” / “SMTP error”  \n     • “réception bloquée” / “IMAP timeout”  \n     • “redirection mail” / “forward étudiant”  \n   Thématiques RAG suggérées :  \n     – Configuration de la messagerie (IMAP/SMTP, Exchange)  \n     – Webmail via ENT  \n     – Redirection des mails étudiants  \n\n5. Imprimantes et périphériques  \n   Mots-clés/phrases :  \n     • “imprimante non trouvée” / “erreur spooler”  \n     • “connexion USB/ réseau”  \n     • “driver manquant”  \n   Thématiques RAG suggérées :  \n     – Mise à jour des mots de passe d’imprimante (Gestionnaire d’identification)  \n     – Installation et partage d’imprimantes sur Windows  \n     – Dépannage spooler  \n\n6. Performance et lenteur  \n   Mots-clés/phrases :  \n     • “ordinateur lent” / “démarrage trop long”  \n     • “applications réagissent mal”  \n     • “goulot d’étranglement réseau”  \n   Thématiques RAG suggérées :  \n     – Agents de sécurité et inventaire (OCS, Cortex XDR)  \n     – Analyse de charge réseau / débogage DNS  \n     – Vérification des services et mises à jour  \n\n7. Sécurité et antivirus  \n   Mots-clés/phrases :  \n     • “alerte virus” / “malware detecté”  \n     • “pare-feu bloque”  \n     • “posture VPN”  \n   Thématiques RAG suggérées :  \n     – Installation et configuration de Cortex XDR  \n     – GlobalProtect : posture et remontées  \n     – Bonnes pratiques de sécurité  \n\n8. Téléphonie et messagerie vocale  \n   Mots-clés/phrases :  \n     • “pas de tonalité” / “pas d’appel”  \n     • “renvoi d’appel” / “messagerie vocale”  \n     • “conférence à 3” / “parking d’appel”  \n   Thématiques RAG suggérées :  \n     – Guide Téléphonie IP (codes fonctions, conf call)  \n     – Numérotation internes/externe  \n     – Paramètres code de sécurité et messagerie  \n\nChaque fois qu’une plainte ou un mot-clé est détecté, le système RAG peut renvoyer :  \n • Le document ou la section précise à consulter  \n • Un diagnostic automatisé (checklist de vérifications)  \n • Des FAQ ou didacticiels associés  \n • Des liens vers les guides de l’ENT ou le portail 5000.'        - 
    """),
    HumanMessage(content="format = {{document_id: document}}\n" + str({i: d for i, d in enumerate(result.documents)})),
]

keyword_mappings: Keywords = LLM.with_structured_output(Keywords).invoke(messages)
keyword_mappings.mapping

[KeywordMapping(document_id=0, resume='Guide de changement de mot de passe UTC et recommandations ANSSI pour créer un mot de passe robuste et sécurisé.', mots_cles=['mot-de-passe-oublie', 'identifiants-invalides']),
 KeywordMapping(document_id=1, resume='Instructions pour modifier le mot de passe de session Windows et mettre à jour le Gestionnaire d’identification pour imprimantes et partages réseau.', mots_cles=['mot-de-passe-oublie', 'identifiants-invalides', 'lecteur-reseau-inaccessible']),
 KeywordMapping(document_id=2, resume='Configuration des serveurs IMAP/SMTP et accès Webmail via ENT, avec guides pour Outlook et Thunderbird.', mots_cles=['envoi-mail-echoue', 'smtp-error', 'reception-bloquee', 'imap-timeout']),
 KeywordMapping(document_id=3, resume='Paramétrage et dépannage des connexions Wi-Fi (UTC, eduroam, invités) et filaire (RJ45, déclaration MAC, DNS).', mots_cles=['connexion-eduroam', 'pas-d-internet', 'aucun-reseau', 'deconnecte']),
 KeywordMapping(document_id=4, resume

## 5 - Sauvegarde de la BDD

In [8]:
from inferers.json_db import JSONKeywordDB

all_keywords = set()
for km in keyword_mappings.mapping:
    all_keywords.update(km.mots_cles)

DB = JSONKeywordDB(
    keywords = {k: "" for k in all_keywords},
    documents = {doc: km.mots_cles for doc, km in zip(DOCS, keyword_mappings.mapping)},
)

with open("data/index/db_1.json", "w+", encoding="utf-8") as f:
    f.write(DB.model_dump_json())
    
with open("data/index/keyword_prompt_1.json", "w+", encoding="utf-8") as f:
    f.write(keyword_prompt)

# Inférence

In [9]:
from typing import List
import json, pydantic
from langchain_core.messages import SystemMessage

with open("data/index/db_1.json", "r", encoding="utf-8") as f:
    db = json.load(f)
    
with open("data/index/keyword_prompt_1.json", "r", encoding="utf-8") as f:
    keyword_prompt = f.read()
    
class KW(pydantic.BaseModel):
    keywords: List[str]

PROMPT = [
    SystemMessage(
        content="Sélectionne tous les mots clés qui correspondent à peu près à la situation de l'utilisateur d'après la conversation."
                + "\nSélectionne uniquement les mots clés parmi la liste suivante:"
                + str(list(db["keywords"].keys()))
                + "\n\nVoici quelques informations additionnelles sur les mots clés et les informations disponibles:\n"
                + keyword_prompt
    )
]
PROMPT[0].content

"Sélectionne tous les mots clés qui correspondent à peu près à la situation de l'utilisateur d'après la conversation.\nSélectionne uniquement les mots clés parmi la liste suivante:['montage-smb-echoue', 'sftp', 'pas-de-tonalite', 'pare-feu-bloque', 'forward-etudiant', 'deconnecte', 'mot-de-passe-oublie', 'identifiants-invalides', 'aucun-reseau', 'parking-d-appel', 'renvoi-d-appel', 'erreur-openvpn', 'messagerie-vocale', 'posture-vpn', 'reception-bloquee', 'pas-d-appel', 'redirection-mail', 'imap-timeout', 'vpn-ne-demarre-pas', 'alerte-virus', 'lecteur-reseau-inaccessible', 'malware-detecte', 'conference-a-3', 'envoi-mail-echoue', 'pas-d-internet', 'televersement-impossible', 'globalprotect', 'connexion-eduroam', 'smtp-error']\n\nVoici quelques informations additionnelles sur les mots clés et les informations disponibles:\nVoici une proposition d’une quarantaine de mots-clés (format lower+dash) couvrant les plaintes/types de demandes que les usagers peuvent adresser au service informati

In [10]:
kw = LLM.with_structured_output(KW).invoke(PROMPT + [HumanMessage(content="Je n'arrive pas à me connecter à mon compte, j'ai oublié mon mot de passe.")])

chunks = set()
for k in kw.keywords:
    chunks.update([doc for doc, keywords in db["documents"].items() if k in keywords])
chunks

{"Changer le mot de passe UTC : https://comptes.utc.fr/accounts-web/tools/changePassword.xhtml (outil officiel). Recommandations ANSSI (http://www.ssi.gouv.fr/IMG/pdf/NP_MDP_NoteTech.pdf) :\n* 8–16 caractères : majuscules, minuscules, chiffres et symboles autorisés (. / * etc.), mais interdits < > ; ` : & % * ? / \\ { } |.\n* Ni mot du dictionnaire, ni nom propre ou marque, ni élément d'identité (nom, date), ni exemple internet.\n* Ne jamais écrire, stocker ni envoyer par mail.\n\nConseils : choisir de préférence 8 caractères, aléatoires ou mnémoniques (phrase personnelle, premières lettres). Garder le secret.",
 'Changer le mot de passe de session Windows : appuyer simultanément Ctrl+Alt+Suppr (voir docs 845/846/847). Saisir l’ancien et le nouveau mot de passe, valider.\n\nMettre à jour le mot de passe aux imprimantes et partages réseau (stocké localement dans le Gestionnaire d’identification) :\n1. Panneau de configuration > Gestionnaire d’identification (doc 848/849).\n2. Sélectionn